### Order By
Use aggregate functions like COUNT(), along with GROUP BY to treat multiple rows as a single group.

ORDER BY is usually the last clause in the query, and it sorts the results returned by the rest of your query.

In [1]:
from google.cloud import bigquery
import numpy as np
import pandas as pd
import os

In [2]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/Users/xlyue/Documents/kaggle learning/Intro to SQL/sql-bq-244804-803e47cbe74e.json"

#### Dates
There are two ways that dates can be stored in BigQuery: as a `DATE` or as a `DATETIME`.

#### Extract
Often you'll want to look at part of a date, like the year or the day. You can do this with `EXTRACT`.

In [3]:
client = bigquery.Client()
dataset_ref = client.dataset('nhtsa_traffic_fatalities', project = 'bigquery-public-data')
# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

In [4]:
tables = list(client.list_tables(dataset))
for table in tables:
    print(table.table_id)
print()
print(len(tables))

accident_2015
accident_2016
cevent_2015
cevent_2016
damage_2015
damage_2016
distract_2015
distract_2016
drimpair_2015
drimpair_2016
factor_2015
factor_2016
maneuver_2015
maneuver_2016
nmcrash_2015
nmcrash_2016
nmimpair_2015
nmimpair_2016
nmprior_2015
nmprior_2016
parkwork_2015
parkwork_2016
pbtype_2015
pbtype_2016
person_2015
person_2016
safetyeq_2015
safetyeq_2016
vehicle_2015
vehicle_2016
vevent_2015
vevent_2016
vindecode_2015
vindecode_2016
violatn_2015
violatn_2016
vision_2015
vision_2016
vsoe_2015
vsoe_2016

40


In [4]:
# Construct a reference to the "accident_2015" table
table_ref = dataset_ref.table('accident_2015')
table = client.get_table(table_ref)

# Preview the first five lines of the "accident_2015" table
client.list_rows(table, max_results=5).to_dataframe()

,state_number,state_name,consecutive_number,number_of_vehicle_forms_submitted_all,number_of_motor_vehicles_in_transport_mvit,number_of_parked_working_vehicles,number_of_forms_submitted_for_persons_not_in_motor_vehicles,number_of_persons_not_in_motor_vehicles_in_transport_mvit,number_of_persons_in_motor_vehicles_in_transport_mvit,number_of_forms_submitted_for_persons_in_motor_vehicles,...,minute_of_ems_arrival_at_hospital,related_factors_crash_level_1,related_factors_crash_level_1_name,related_factors_crash_level_2,related_factors_crash_level_2_name,related_factors_crash_level_3,related_factors_crash_level_3_name,number_of_fatalities,number_of_drunk_drivers,timestamp_of_crash
0,19,Iowa,190204,1,1,0,0,0,1,1,...,2,0,None,0,None,0,None,1,1,2015-09-11 20:20:00+00:00
1,19,Iowa,190233,1,1,0,0,0,1,1,...,88,0,None,0,None,0,None,1,1,2015-11-01 00:30:00+00:00
2,19,Iowa,190179,1,1,0,0,0,2,2,...,1,0,None,0,None,0,None,1,0,2015-05-04 16:18:00+00:00
3,19,Iowa,190248,1,1,0,0,0,4,4,...,99,0,None,0,None,0,None,2,0,2015-11-17 12:26:00+00:00
4,19,Iowa,190231,1,1,0,0,0,1,1,...,88,0,None,0,None,0,None,1,0,2015-10-31 04:49:00+00:00


Extract the day data from timestamp column and group by the day of the week, then order by the days with most accidents.

(That is the day of a week not the day data in the timestamp)

In [5]:
query = """
           SELECT COUNT(consecutive_number) AS num_accidents, 
           EXTRACT(DAYOFWEEK FROM timestamp_of_crash) AS day_of_week
           FROM `bigquery-public-data.nhtsa_traffic_fatalities.accident_2015`
           GROUP BY day_of_week
           ORDER BY num_accidents DESC
           """

In [6]:
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed = 10**9)
query_job = client.query(query, job_config = safe_config)

In [7]:
# API request - run the query, and convert the results to a pandas DataFrame
accidents_by_day = query_job.to_dataframe()

# Print the DataFrame
accidents_by_day

,num_accidents,day_of_week
0,5659,7
1,5298,1
2,4916,6
3,4460,5
4,4182,4
5,4038,2
6,3985,3


Here, the above table shows the index of `day_of_week`, 1 means Sunday and 7 means Saturday.

Exercises

In [8]:
client = bigquery.Client()
dataset_ref = client.dataset('world_bank_intl_education', project='bigquery-public-data')

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

In [9]:
# Construct a reference to the "international_education" table
table_ref = dataset_ref.table('international_education')

# API request - fetch the table
table = client.get_table(table_ref)

# Preview the first five lines of the "international_education" table
client.list_rows(table, max_results=5).to_dataframe()

,country_name,country_code,indicator_name,indicator_code,value,year
0,Lebanon,LBN,Official entrance age to pre-primary education...,UIS.THAGE.0,3.000000e+00,2016
1,Lebanon,LBN,"Unemployment, total (% of total labor force)",SL.UEM.TOTL.ZS,6.500000e+00,2016
2,Liberia,LBR,Population of the official age for secondary e...,SP.SEC.TOTL.FE.IN,3.082780e+05,2016
3,Sub-Saharan Africa (excluding high income),SSA,"Population, total",SP.POP.TOTL,1.033011e+09,2016
4,Upper middle income,UMC,"Population, ages 0-14 (% of total)",SP.POP.0014.TO.ZS,2.059912e+01,2016


We focus on `indicator_code` column

Which countries spend the largest fraction of GDP on education?  

In [10]:
country_spend_pct_query = """
                                       SELECT country_name, AVG(value) AS avg_ed_spending_pct
                                       FROM `bigquery-public-data.world_bank_intl_education.international_education`
                                       WHERE indicator_code = 'SE.XPD.TOTL.GD.ZS' and year >= 2010 and year <= 2017
                                       GROUP BY country_name
                                       ORDER BY avg_ed_spending_pct DESC
                                       """

# Set up the query (cancel the query if it would use too much of 
# your quota, with the limit set to 1 GB)
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed = 10**9)
country_spend_pct_query_job = client.query(country_spend_pct_query, job_config = safe_config)

# API request - run the query, and return a pandas DataFrame
country_spending_results = country_spend_pct_query_job.to_dataframe()

In [11]:
# View top few rows of results
print(country_spending_results.head())

            country_name  avg_ed_spending_pct
0                   Cuba            12.837270
1  Micronesia, Fed. Sts.            12.467750
2        Solomon Islands            10.001080
3                Moldova             8.372153
4                Namibia             8.349610


Identifying `indicator_code` for more than 175 records of countries

In [13]:
code_count_query = """
                             select indicator_code, indicator_name, count(1) as num_rows
                             from `bigquery-public-data.world_bank_intl_education.international_education`
                             where year = 2016
                             group by indicator_code, indicator_name
                             having num_rows >= 175
                             order by num_rows desc
                             """
# Here count(*) * can be 1 or indicator_code

# Set up the query
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed = 10**9)
code_count_query_job = client.query(code_count_query, job_config = safe_config)

# API request - run the query, and return a pandas DataFrame
code_count_results = code_count_query_job.to_dataframe()

# View top few rows of results
print(code_count_results.head())

      indicator_code                       indicator_name  num_rows
0        SP.POP.TOTL                    Population, total       232
1        SP.POP.GROW         Population growth (annual %)       232
2     IT.NET.USER.P2      Internet users (per 100 people)       223
3  SP.POP.1564.TO.ZS  Population, ages 15-64 (% of total)       213
4  SP.POP.0014.FE.IN        Population, ages 0-14, female       213


In [14]:
code_count_query = """
                             select indicator_code, indicator_name, count(indicator_code) as num_rows
                             from `bigquery-public-data.world_bank_intl_education.international_education`
                             where year = 2016
                             group by indicator_code, indicator_name
                             having num_rows >= 175
                             order by num_rows desc
                             """
# Here count(*) * can be 1 or indicator_code

# Set up the query
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed = 10**9)
code_count_query_job = client.query(code_count_query, job_config = safe_config)

# API request - run the query, and return a pandas DataFrame
code_count_results = code_count_query_job.to_dataframe()

# View top few rows of results
print(code_count_results.head())

      indicator_code                      indicator_name  num_rows
0        SP.POP.TOTL                   Population, total       232
1        SP.POP.GROW        Population growth (annual %)       232
2     IT.NET.USER.P2     Internet users (per 100 people)       223
3  SP.POP.0014.TO.ZS  Population, ages 0-14 (% of total)       213
4  SP.POP.1564.MA.IN        Population, ages 15-64, male       213
